<h1 style="font-size:3rem;color:blue;">Analisando Jogos NBA 2023</h1>

Coletando dados do banco de dados.

In [ ]:
import mysql.connector as connection
import pandas as pd
import math

try:
    mydb = connection.connect(host="localhost", database = 'nbaonnumbers',user="sa", passwd="RomuloJalim70",use_pure=True)
    query = "select id_game as 'id', date, visitor.full_name as 'visitor', home.full_name as 'home', visit_score, home_score, visitor.division as 'visitor_division', home.division as 'home_division', visitor.conference as 'visitor_conference', home.conference as 'home_conference' from game inner join team visitor on visitor.id_team = game.id_visit_team inner join team home on home.id_team = game.id_home_team where game.league = 'NBA';"
    result_dataFrame = pd.read_sql(query, mydb)
    mydb.close()
except Exception as e:
    mydb.close()
    print(str(e))

Revendo dados de uma linha da tabela

In [ ]:
print(result_dataFrame.iloc[0])

Setando o vencedor dos jogos

In [ ]:
teamwinner = []
for index, row in result_dataFrame.iterrows():
    if int(row['visit_score']) > int(row['home_score']):
        teamwinner.append("VISIT")
    elif int(row['visit_score']) < int(row['home_score']):
        teamwinner.append("HOME")
    else:
        teamwinner.append("NA")
result_dataFrame = result_dataFrame.assign(winner=teamwinner)

In [ ]:
valuescounted = result_dataFrame['winner'].value_counts()
print('Percent home ' + str(100*(valuescounted['HOME']/(valuescounted['HOME']+valuescounted['VISIT']))))
print('Percent visit ' + str(100*(valuescounted['VISIT']/(valuescounted['HOME']+valuescounted['VISIT']))))

Totals e diferenças de pontos

In [ ]:
totals = []
pointsDiff = []
for index, row in result_dataFrame.iterrows():
    totals.append(int(row['visit_score']) + int(row['home_score']))
    pointsDiff.append(abs(int(row['visit_score']) - int(row['home_score'])))
    
result_dataFrame = result_dataFrame.assign(total_points=totals)
result_dataFrame = result_dataFrame.assign(diff_points=pointsDiff)

In [ ]:
print("Describe total de pontos ")
result_dataFrame.query("visit_score != 0")['total_points'].describe()

In [ ]:
print("Describe da diferença de pontos da liga ")
result_dataFrame.query("visit_score != 0")['diff_points'].describe()

Construindo matrix de jogos e diferença de Vitórias x Derrotas para calcular ranking.

In [ ]:
from resources.nba_statistics import nbaStatistics

Only_played_games = result_dataFrame.query("visit_score != 0")

d = {'name':['Boston Celtics', 'Brooklyn Nets', 'New York Knicks', 'Philadelphia 76ers', 'Toronto Raptors',
                        'Chicago Bulls', 'Cleveland Cavaliers', 'Detroit Pistons', 'Indiana Pacers', 'Milwaukee Bucks',
                        'Atlanta Hawks', 'Charlotte Hornets', 'Miami Heat', 'Orlando Magic', 'Washington Wizards',
                        'Denver Nuggets', 'Minnesota Timberwolves', 'Oklahoma City Thunder', 'Portland Trail Blazers', 'Utah Jazz',
                        'Golden State Warriors', 'Los Angeles Clippers', 'Los Angeles Lakers', 'Phoenix Suns', 'Sacramento Kings',
                        'Dallas Mavericks', 'Houston Rockets', 'Memphis Grizzlies', 'New Orleans Pelicans', 'San Antonio Spurs'], 
            'record':[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
            'jogos':[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]}      

df = pd.DataFrame(d)

for index, row in Only_played_games.iterrows():
    if row['winner'] == "HOME":
        df.loc[df.name == row['visitor'], 'record'] = df.loc[df.name == row['visitor'], 'record'] - 1
        df.loc[df.name == row['home'], 'record'] = df.loc[df.name == row['home'], 'record'] + 1
    elif row['winner'] == "VISIT":
        df.loc[df.name == row['home'], 'record'] = df.loc[df.name == row['home'], 'record'] - 1
        df.loc[df.name == row['visitor'], 'record'] = df.loc[df.name == row['visitor'], 'record'] + 1
    
    df.loc[df.name == row['home'], 'jogos'] = df.loc[df.name == row['home'], 'jogos'] + 1
    df.loc[df.name == row['visitor'], 'jogos'] = df.loc[df.name == row['visitor'], 'jogos'] + 1

In [ ]:

from resources.nba_statistics import nbaStatistics
from sympy import symbols, Eq, solve

sol = nbaStatistics().nbaRank(df)
sol.loc[sol.teamName == 'Boston Celtics', 'teamRank']

homeRank = []
visitorRank = []
diffRank = []
bestRankWins = []
for index, row in Only_played_games.iterrows():
    homeRanking = float(sol.loc[sol.teamName == row['home'], 'teamRank'])/1000
    visitorRanking = float(sol.loc[sol.teamName == row['visitor'], 'teamRank'])/1000
    homeRank.append(homeRanking)
    visitorRank.append(visitorRanking)
    diffRank.append(abs(homeRanking - visitorRanking))
    
    if row['winner'] == "HOME" and homeRanking > visitorRanking:
        bestRankWins.append("SIM")
    elif row['winner'] == "VISIT" and homeRanking < visitorRanking:
        bestRankWins.append("SIM")
    else:
        bestRankWins.append("NAO")
    
Only_played_games = Only_played_games.assign(home_rank = homeRank)
Only_played_games = Only_played_games.assign(visitor_rank = visitorRank)
Only_played_games = Only_played_games.assign(diff_rank = diffRank)
Only_played_games = Only_played_games.assign(best_rank_wins = bestRankWins)

<h2 style="font-size:3rem;color:blue;">Time Com melhor Ranking</h2>

In [ ]:
print("Describe melhor ranking ganha ? ")
valuescountedbestrank = Only_played_games['best_rank_wins'].value_counts()
print('Percent sim ' + str(100*(valuescountedbestrank['SIM']/(valuescountedbestrank['SIM']+valuescountedbestrank['NAO']))))
print('Percent nao ' + str(100*(valuescountedbestrank['NAO']/(valuescountedbestrank['NAO']+valuescountedbestrank['SIM']))))

In [ ]:
print("Describe melhor ranking ganha pela diferença de rank ")
Only_played_games.query("best_rank_wins == 'SIM'")['diff_rank'].describe()

In [ ]:
print("Describe melhor ranking ganha pela diferença de rank maior que 1.0 ")
Only_played_games.query("best_rank_wins == 'SIM'").query("diff_rank <= 1.0")['diff_rank'].describe()

In [ ]:
print("Describe rank melhor ganhou e com diferença de rank maior que 1.0 ")
#Only_played_games.query("best_rank_wins == 'SIM'").query("diff_rank >= 1.0")['diff_rank'].describe()
valuescountedbestrank2 = Only_played_games.query("diff_rank >= 1.0")['best_rank_wins'].value_counts()
print(valuescountedbestrank2['SIM'])
print(valuescountedbestrank2['NAO'])
print('Percent sim com rank > 1 e ' + str(100*(valuescountedbestrank2['SIM']/(valuescountedbestrank2['SIM']+valuescountedbestrank2['NAO']))))
print('Percent nao com rank > 1 e ' + str(100*(valuescountedbestrank2['NAO']/(valuescountedbestrank2['NAO']+valuescountedbestrank2['SIM']))))

<h2 style="font-size:3rem;color:blue;">Back to back games logic</h2>

In [ ]:
import datetime as dt
import numpy as np

datas = {'name':['Boston Celtics', 'Brooklyn Nets', 'New York Knicks', 'Philadelphia 76ers', 'Toronto Raptors',
                        'Chicago Bulls', 'Cleveland Cavaliers', 'Detroit Pistons', 'Indiana Pacers', 'Milwaukee Bucks',
                        'Atlanta Hawks', 'Charlotte Hornets', 'Miami Heat', 'Orlando Magic', 'Washington Wizards',
                        'Denver Nuggets', 'Minnesota Timberwolves', 'Oklahoma City Thunder', 'Portland Trail Blazers', 'Utah Jazz',
                        'Golden State Warriors', 'Los Angeles Clippers', 'Los Angeles Lakers', 'Phoenix Suns', 'Sacramento Kings',
                        'Dallas Mavericks', 'Houston Rockets', 'Memphis Grizzlies', 'New Orleans Pelicans', 'San Antonio Spurs'], 
            'ultimoJogo':[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]}      

default_date = dt.datetime(2023, 6, 1, 0,0,0)
datas_df = pd.DataFrame(datas)
datas_df = datas_df.replace(0,default_date)

daysLastGameVisit = []
daysLastGameHome = []
for index, row in Only_played_games.iterrows():
    data_jogo = dt.datetime.strptime(str(row['date']),"%Y-%m-%d %H:%M:%S")
    home_index = datas_df.loc[df.name == row['home'], 'ultimoJogo'].index[0]
    data_home = dt.datetime.strptime(str(datas_df.iloc[home_index]['ultimoJogo']), "%Y-%m-%d %H:%M:%S")
    visitor_index = datas_df.loc[df.name == row['visitor'], 'ultimoJogo'].index[0]
    data_visitor = dt.datetime.strptime(str(datas_df.iloc[visitor_index]['ultimoJogo']), "%Y-%m-%d %H:%M:%S")
    
    delta = data_jogo - data_home
    daysLastGameHome.append(delta.days)
    datas_df.loc[datas_df.name == row['home'], 'ultimoJogo'] = data_jogo

    delta = data_jogo - data_visitor
    daysLastGameVisit.append(delta.days)
    datas_df.loc[datas_df.name == row['visitor'], 'ultimoJogo'] = data_jogo
    
Only_played_games = Only_played_games.assign(days_last_game_visit = daysLastGameVisit)
Only_played_games = Only_played_games.assign(days_last_game_home = daysLastGameHome)

Diferença de dias de descanso para o vencedor

In [ ]:
diffRestDaysToWinner = []
for index, row in Only_played_games.iterrows():
    if row['winner'] == "HOME":
        diffRestDaysToWinner.append(abs(row['days_last_game_home'] - row['days_last_game_visit']))
    else:
        diffRestDaysToWinner.append(abs(row['days_last_game_visit'] - row['days_last_game_home']))

Only_played_games = Only_played_games.assign(diff_days_rest_winner = diffRestDaysToWinner)

In [ ]:
Only_played_games.query("diff_rank >= 1.0").query("best_rank_wins == 'NAO'")['diff_days_rest_winner'].describe()

Mesma conferencia e divisão

In [ ]:
sameConference = []
sameDivision = []
for index, row in Only_played_games.iterrows():
    if row['visitor_conference'] == row['home_conference']:
        sameConference.append("SIM")
    else:
        sameConference.append("NAO")
    
    if row['visitor_division'] == row['home_division']:
        sameDivision.append("SIM")
    else:
        sameDivision.append("NAO")

Only_played_games = Only_played_games.assign(same_conference = sameConference)
Only_played_games = Only_played_games.assign(same_division = sameDivision)

In [ ]:
print("Rank >=1 com o favorito perdendo, levantamento de jogos mesma conferencia e divisao")
valuescounted = Only_played_games.query("diff_rank >= 1.0").query("best_rank_wins == 'NAO'")['same_division'].value_counts()
print(valuescounted['SIM'])
print(valuescounted['NAO'])
print('Mesma Divisao ' + str(100*(valuescounted['SIM']/(valuescounted['SIM']+valuescounted['NAO']))))
print('Diferente Divisao ' + str(100*(valuescounted['NAO']/(valuescounted['NAO']+valuescounted['SIM']))))

In [ ]:
print("Na mesma divisao, levantamento de jogos mesma conferencia e divisao")
valuescounted = Only_played_games.query("same_division == 'SIM'")['best_rank_wins'].value_counts()
print(valuescounted['SIM'])
print(valuescounted['NAO'])
print('Mesma Divisao ' + str(100*(valuescounted['SIM']/(valuescounted['SIM']+valuescounted['NAO']))))
print('Diferente Divisao ' + str(100*(valuescounted['NAO']/(valuescounted['NAO']+valuescounted['SIM']))))